In [32]:
import json
import re

In [10]:
with open('../data/evaluate_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

print(data)

[{'task_id': 't00', 'task_description': 'Open Watches menu and retrieve the name of the first product in the list.', 'task_type': 'lookup', 'start_url': '__SHOPPING__', 'require_login': False, 'eval': {'eval_type': ['string_match', 'regex_match', 'url_match', 'dom_match'], 'string_match': {'match_type': 'exact', 'match_value': 'Aim Analog Watch'}, 'regex_match': {'email': {'pattern': '^[\\w\\.-]+@[\\w\\.-]+\\.\\w{2,}$', 'required': 'true', 'description': 'A valid email address format (e.g., user@example.com)'}, 'password': {'pattern': '^(?=.*[A-Z])(?=.*[a-z])(?=.*\\d)[A-Za-z\\d!@#$%^&*()_+=\\-]{8,}$', 'required': 'true', 'description': 'Minimum 8 characters, at least one uppercase letter, one lowercase letter, and one number'}}, 'url_match': {'url': 'last', 'match_type': 'contains', 'match_value': 'customer/account'}, 'dom_match': {'url': 'http://localhost:8084/aim-analog-watch.html', 'dom_extractor': "document.querySelector('#old-price-36 .price').innerText;", 'match_type': 'exact', '

In [29]:
task_id_to_find = 't00'  # Example task ID to find
# Tìm task
task = next((task for task in data if task['task_id'] == task_id_to_find), None)
if task:
    print(task)
else:
    print("Task not found")
    exit(1)
print(task['eval'])

{'task_id': 't00', 'task_description': 'Open Watches menu and retrieve the name of the first product in the list.', 'task_type': 'lookup', 'start_url': '__SHOPPING__', 'require_login': False, 'eval': {'eval_type': ['string_match', 'regex_match', 'url_match', 'dom_match'], 'string_match': {'match_type': 'exact', 'match_value': 'Aim Analog Watch'}, 'regex_match': {'email': {'pattern': '^[\\w\\.-]+@[\\w\\.-]+\\.\\w{2,}$', 'required': 'true', 'description': 'A valid email address format (e.g., user@example.com)'}, 'password': {'pattern': '^(?=.*[A-Z])(?=.*[a-z])(?=.*\\d)[A-Za-z\\d!@#$%^&*()_+=\\-]{8,}$', 'required': 'true', 'description': 'Minimum 8 characters, at least one uppercase letter, one lowercase letter, and one number'}}, 'url_match': {'url': 'last', 'match_type': 'contains', 'match_value': 'customer/account'}, 'dom_match': {'url': 'http://localhost:8084/aim-analog-watch.html', 'dom_extractor': "document.querySelector('#old-price-36 .price').innerText;", 'match_type': 'exact', 'm

In [ ]:
agent_result = task['eval']['agent_result']
def string_match(agent_result, match_value):
    return True
def regex_match(agent_result, match_value):
    return True
def url_match(agent_result, match_value):
    return True    
def dom_match(agent_result, match_value):
    return True

result = False
for match_type in task['eval']['eval_type']:
    print(f"Match type: {match_type}")
    flag_type = False

    if match_type == 'string_match':
        flag_type = True
        if not string_match(agent_result, task['eval']['match_value']):
            break
        result = True

    elif match_type == 'regex_match':
        flag_type = True
        if not regex_match(agent_result, task['eval']['match_value']):
            break
        result = True

    elif match_type == 'url_match':
        flag_type = True
        if not url_match(agent_result, task['eval']['match_value']):
            break
        result = True

    elif match_type == 'dom_match':
        flag_type = True
        if not dom_match(agent_result, task['eval']['match_value']):
            break
        result = True

    else:
        print("This is an unknown evaluation task type.")

Match type: string_match
This is a string match evaluation task.
Match type: regex_match
This is a regex match evaluation task.
Match type: url_match
This is a URL match evaluation task.
Match type: dom_match
This is a DOM match evaluation task.


In [ ]:
# Giả sử đây là agent_result bạn lấy được từ UI
# (bạn cần thay giá trị thực tế)
agent_result = {
    "string_match": "Aim Analog Watch",
    "url_match": "http://localhost:8084/customer/account",
    "dom_match": "$45.00",
    "regex_match": {
        "email": "user@example.com",
        "password": "MySecurePass1"
    }
}

# Hàm kiểm tra string_match
def string_match(agent_value, match_conf):
    if isinstance(match_conf['match_value'], str):
        match_values = [match_conf['match_value']]
    else:
        match_values = match_conf['match_value']

    if match_conf['match_type'] == 'exact':
        return all(agent_value == v for v in match_values)
    elif match_conf['match_type'] == 'contains':
        return all(v in agent_value for v in match_values)
    else:
        print("Unknown string_match type")
        return False

# Hàm kiểm tra regex_match
def regex_match(agent_value, match_conf):
    for field, conf in match_conf.items():
        value = agent_value.get(field, "")
        pattern = conf['pattern']
        if not re.match(pattern, value):
            print(f"Regex match failed for {field}: {value} does not match {pattern}")
            return False
    return True

# Hàm kiểm tra url_match
def url_match(agent_value, match_conf):
    if match_conf['match_type'] == 'exact':
        return agent_value == match_conf['match_value']
    elif match_conf['match_type'] == 'contains':
        return match_conf['match_value'] in agent_value
    else:
        print("Unknown url_match type")
        return False

# Hàm kiểm tra dom_match
def dom_match(agent_value, match_conf):
    if match_conf['match_type'] == 'exact':
        return agent_value == match_conf['match_value']
    elif match_conf['match_type'] == 'contains':
        return match_conf['match_value'] in agent_value
    else:
        print("Unknown dom_match type")
        return False

In [ ]:
# Hàm đánh giá 1 task
def evaluate_task(task, agent_result):
    handlers = {
        'string_match': string_match,
        'regex_match': regex_match,
        'url_match': url_match,
        'dom_match': dom_match
    }

    eval_block = task['eval']
    result = False

    for eval_type in eval_block['eval_type']:
        print(f"Evaluating {eval_type}...")
        handler = handlers.get(eval_type)
        if handler:
            agent_value = agent_result[eval_type]
            match_conf = eval_block[eval_type]

            if not handler(agent_value, match_conf):
                print(f"Evaluation failed for {eval_type}")
                return False  # Nếu 1 điều kiện fail → fail toàn bộ
            result = True
        else:
            print(f"Unknown eval type: {eval_type}")

    return result